In [3]:
# breakhistf2v6.ipynb  v1.0.0 
# 
#  --------------------------------------------------
#  Hangzhou Domain Zones Technology Co., Ltd

#  Apache Licence 2.0       https://www.apache.org/licenses/LICENSE-2.0
#  --------------------------------------------------


import os
import tensorflow as tf
import numpy as np
import csv
import matplotlib.pyplot as plt
from tensorflow import keras
import neural_structured_learning as nsl
from tensorflow.python.keras.api._v2.keras import layers, optimizers, losses
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard

tf.random.set_seed(22)
np.random.seed(22)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
assert tf.__version__.startswith('2.')
  
%matplotlib inline

def read_csv(csvnamepath, filename):
     

    # read from csv file
    images, labels = [], []
    with open(os.path.join(csvnamepath, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)

            images.append(img)
            labels.append(label)

    assert len(images) == len(labels)

    return images, labels

 
def load_breakhis(dirc, mode='train'):
    classdir2label={}
    filename="tf2breakhisCSV"
    filedirs = os.listdir( dirc)
    for filedir in filedirs:
        if not os.path.isdir(os.path.join(dirc,filedir)):
            continue
        classdir2label[filedir]=len(classdir2label.keys())
    

     
    images, labels = read_csv(os.path.join(os.path.abspath('.'),'tf2breakhis'), filename )  
    if mode == 'train':  # 60%
        images = images[:int(0.6 * len(images))]
        labels = labels[:int(0.6 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # 20% = 80%->100%
        images = images[int(0.8 * len(images)):]
        labels = labels[int(0.8 * len(labels)):]
    return images, labels, classdir2label
 
img_mean = tf.constant([0.485, 0.456, 0.406])
img_std = tf.constant([0.229, 0.224, 0.225])

@tf.function
def normalize(x, mean=img_mean, std=img_std):
    # x shape: [224, 224, 3]
    # mean：shape为1；这里用到了广播机制。我们安装好右边对齐的原则，可以得到如下；
    # mean : [1, 1, 3], std: [3]        先插入1
    # mean : [224, 224, 3], std: [3]    再变为224
    x = (x - mean)/std
    return x

# 数据normalize之后，这里有一个反normalizaion操作。比如数据可视化的时候，需要反过来。
@tf.function
def denormalize(x, mean=img_mean, std=img_std):
    x = x * std + mean
    return x

 


# 预处理的函数，复制过来。
@tf.function
def preprocess(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
   
    x = tf.image.random_flip_left_right(x)
    #plt.imshow(x)
  
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
    #plt.imshow(x)
    
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y


# 预处理的函数，复制过来。
@tf.function
def preprocess1(x,y):
    # x: 图片的路径，y：图片的数字编码
   
    x = tf.io.read_file(x)
     
    x = tf.image.decode_jpeg(x, channels=3) # RGBA
    
    x = tf.image.resize(x, [244, 244])
   
    
    # x = tf.image.random_flip_up_down(x)
    x = tf.image.random_crop(x, [224,224,3])
      
    # x: [0,255]=> -1~1
    x = tf.cast(x, dtype=tf.float32) / 255.
    x = normalize(x)
    y = tf.convert_to_tensor(y)
    y = tf.one_hot(y, depth=8)

    return x, y




 

batchsz = 32
epochnum =10
# creat train db   一般训练的时候需要shuffle。其它是不需要的。
images, labels, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'train')
db_train = tf.data.Dataset.from_tensor_slices((images, labels))  # 变成个Dataset对象。
db_train = db_train.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)  # map函数图片路径变为内容。
 

# crate validation db
images2, labels2, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'val')
db_val = tf.data.Dataset.from_tensor_slices((images2, labels2))
db_val = db_val.shuffle(1000).repeat(2).map(preprocess).shuffle(1000).batch(batchsz)
# create test db
images3, labels3, table = load_breakhis(os.path.join(os.path.abspath('.'),'tf2breakhis'), 'test')
db_test = tf.data.Dataset.from_tensor_slices((images3, labels3))
db_test = db_test.map(preprocess1).batch(batchsz)

 
resnetdense = keras.Sequential([
    layers.Conv2D(16,5,3),
    layers.MaxPool2D(3,3),
    layers.ReLU(),
    layers.Conv2D(64,5,3),
    layers.MaxPool2D(2,2),
    layers.ReLU(),
    
    layers.Flatten(),
    #
    layers.Dense(128), 
    layers.Dropout(rate=0.5),
    
    
    
    layers.Dense(64),  
    layers.ReLU(),
    layers.Dense(8)
   ])


 

# 首先创建Resnet18
# resnet = ResNet(8)

resnetdense.build(input_shape=(batchsz, 224, 224, 3))
resnetdense.summary()

# monitor监听器, 连续5个验证准确率不增加，这个事情触发。
 
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    min_delta=0.001,
    patience=100

)

  

 
# 网络的装配。
resnetdense.compile(optimizer=optimizers.Adam(lr=1e-4),
               loss=losses.CategoricalCrossentropy(from_logits=True),
               metrics=['accuracy'])

# 完成标准的train，val, test;
# 标准的逻辑必须通过db_val挑选模型的参数，就需要提供一个earlystopping技术，

LOGDIR='log/breakhistf2v6' 
resnetdense.fit(db_train, validation_data=db_val, 
                        validation_freq=10, epochs=2000, 
                        callbacks=[TensorBoard(log_dir=LOGDIR)])   # 1个epoch验证1次。触发了这个事情，提前停止了。
resnetdense.evaluate(db_test)
 
 

    
    
 

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            multiple                  1216      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_6 (ReLU)               multiple                  0         
_________________________________________________________________
conv2d_5 (Conv2D)            multiple                  25664     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 multiple                  0         
_________________________________________________________________
re_lu_7 (ReLU)               multiple                  0         
_________________________________________________________________
flatten_2 (Flatten)          multiple                 

297/297 [==============================] - 216s 727ms/step - loss: 0.7786 - accuracy: 0.7081
Epoch 59/700
297/297 [==============================] - 218s 735ms/step - loss: 0.7668 - accuracy: 0.7142
Epoch 60/700
297/297 [==============================] - 287s 965ms/step - loss: 0.7585 - accuracy: 0.7160 - val_loss: 0.7956 - val_accuracy: 0.6994
Epoch 61/700
297/297 [==============================] - 215s 725ms/step - loss: 0.7636 - accuracy: 0.7174
Epoch 62/700
297/297 [==============================] - 214s 721ms/step - loss: 0.7509 - accuracy: 0.7177
Epoch 63/700
297/297 [==============================] - 210s 706ms/step - loss: 0.7505 - accuracy: 0.7149
Epoch 64/700
297/297 [==============================] - 215s 725ms/step - loss: 0.7410 - accuracy: 0.7209
Epoch 65/700
297/297 [==============================] - 216s 726ms/step - loss: 0.7328 - accuracy: 0.7274
Epoch 66/700
297/297 [==============================] - 215s 724ms/step - loss: 0.7332 - accuracy: 0.7275
Epoch 67/700
297/

297/297 [==============================] - 247s 832ms/step - loss: 0.4229 - accuracy: 0.8339
Epoch 206/700
297/297 [==============================] - 249s 839ms/step - loss: 0.4046 - accuracy: 0.8426
Epoch 207/700
297/297 [==============================] - 248s 835ms/step - loss: 0.4053 - accuracy: 0.8442
Epoch 208/700
297/297 [==============================] - 251s 845ms/step - loss: 0.4087 - accuracy: 0.8412
Epoch 209/700
297/297 [==============================] - 227s 764ms/step - loss: 0.4067 - accuracy: 0.8371
Epoch 210/700
297/297 [==============================] - 286s 963ms/step - loss: 0.4109 - accuracy: 0.8418 - val_loss: 0.6579 - val_accuracy: 0.7737
Epoch 211/700
297/297 [==============================] - 215s 725ms/step - loss: 0.3961 - accuracy: 0.8459
Epoch 212/700
297/297 [==============================] - 214s 722ms/step - loss: 0.4063 - accuracy: 0.8415
Epoch 213/700
297/297 [==============================] - 212s 714ms/step - loss: 0.4064 - accuracy: 0.8431
Epoch 214

297/297 [==============================] - 214s 721ms/step - loss: 0.3517 - accuracy: 0.8609
Epoch 280/700
297/297 [==============================] - 326s 1s/step - loss: 0.3324 - accuracy: 0.8690 - val_loss: 0.6410 - val_accuracy: 0.7835
Epoch 281/700
297/297 [==============================] - 253s 853ms/step - loss: 0.3404 - accuracy: 0.8659
Epoch 282/700
297/297 [==============================] - 249s 837ms/step - loss: 0.3341 - accuracy: 0.8655
Epoch 283/700
297/297 [==============================] - 243s 818ms/step - loss: 0.3293 - accuracy: 0.8724
Epoch 284/700
297/297 [==============================] - 247s 833ms/step - loss: 0.3353 - accuracy: 0.8677
Epoch 285/700
297/297 [==============================] - 247s 833ms/step - loss: 0.3127 - accuracy: 0.8743
Epoch 286/700
297/297 [==============================] - 221s 743ms/step - loss: 0.3352 - accuracy: 0.8693
Epoch 287/700
297/297 [==============================] - 219s 736ms/step - loss: 0.3367 - accuracy: 0.8700
Epoch 288/70

Epoch 426/700
297/297 [==============================] - 217s 729ms/step - loss: 0.2557 - accuracy: 0.8967
Epoch 427/700
297/297 [==============================] - 215s 725ms/step - loss: 0.2411 - accuracy: 0.9014
Epoch 428/700
297/297 [==============================] - 218s 735ms/step - loss: 0.2599 - accuracy: 0.8964
Epoch 429/700
297/297 [==============================] - 217s 729ms/step - loss: 0.2428 - accuracy: 0.9017
Epoch 430/700
297/297 [==============================] - 283s 953ms/step - loss: 0.2487 - accuracy: 0.9012 - val_loss: 0.6601 - val_accuracy: 0.7816
Epoch 431/700
297/297 [==============================] - 218s 735ms/step - loss: 0.2475 - accuracy: 0.9013
Epoch 432/700
297/297 [==============================] - 219s 736ms/step - loss: 0.2416 - accuracy: 0.9034
Epoch 433/700
297/297 [==============================] - 215s 723ms/step - loss: 0.2550 - accuracy: 0.9026
Epoch 434/700
297/297 [==============================] - 215s 724ms/step - loss: 0.2566 - accuracy: 0.

297/297 [==============================] - 250s 843ms/step - loss: 0.2030 - accuracy: 0.9165
Epoch 573/700
297/297 [==============================] - 247s 833ms/step - loss: 0.1955 - accuracy: 0.9205
Epoch 574/700
297/297 [==============================] - 275s 926ms/step - loss: 0.1968 - accuracy: 0.9223
Epoch 575/700
297/297 [==============================] - 276s 930ms/step - loss: 0.2109 - accuracy: 0.9174
Epoch 576/700
297/297 [==============================] - 318s 1s/step - loss: 0.2025 - accuracy: 0.9211
Epoch 577/700
297/297 [==============================] - 240s 810ms/step - loss: 0.1899 - accuracy: 0.9203
Epoch 578/700
297/297 [==============================] - 250s 841ms/step - loss: 0.1999 - accuracy: 0.9182
Epoch 579/700
297/297 [==============================] - 228s 767ms/step - loss: 0.1952 - accuracy: 0.9188
Epoch 580/700
297/297 [==============================] - 324s 1s/step - loss: 0.1949 - accuracy: 0.9236 - val_loss: 0.7002 - val_accuracy: 0.7946
Epoch 581/700
2

297/297 [==============================] - 220s 741ms/step - loss: 0.1974 - accuracy: 0.9201
Epoch 647/700
297/297 [==============================] - 230s 776ms/step - loss: 0.1879 - accuracy: 0.9221
Epoch 648/700
297/297 [==============================] - 222s 746ms/step - loss: 0.1862 - accuracy: 0.9270
Epoch 649/700
297/297 [==============================] - 221s 743ms/step - loss: 0.1865 - accuracy: 0.9230
Epoch 650/700
297/297 [==============================] - 289s 974ms/step - loss: 0.1748 - accuracy: 0.9307 - val_loss: 0.7245 - val_accuracy: 0.8047
Epoch 651/700
297/297 [==============================] - 223s 749ms/step - loss: 0.1757 - accuracy: 0.9331
Epoch 652/700
297/297 [==============================] - 222s 748ms/step - loss: 0.1890 - accuracy: 0.9264
Epoch 653/700
297/297 [==============================] - 254s 855ms/step - loss: 0.1754 - accuracy: 0.9286
Epoch 654/700
297/297 [==============================] - 245s 826ms/step - loss: 0.1913 - accuracy: 0.9249
Epoch 655

[0.7095902203023434, 0.81289506]